# Review 2 - Tail Recursion and Recursion in Lettuce

## Topic 1- Tail Recursion
### Recursive Functions.

A function $f$ is defined recursively if the body of the definition refers back to $f$.
- It has a base case or termination case
- A recursive call
- Preconditions: A precondition is a constraint that restricts what inputs can be used to call a function.  For instance, the `factorial` function has the precondition that its input must be non negative. In scala we can use the `require` keyword to specify a precondition.


In [1]:
def factorialWithPreconds(x: Int): Int = {
    require(x >= 0) // This is a precondition
    if (x == 0) {
       1 // This is called the base case
    } else { 
       x * factorialWithPreconds(x-1) // The recursion is here.   
    }
}

defined function factorialWithPreconds

In [2]:
val y = factorialWithPreconds(2)

y: Int = 2

### What is wrong with regular recursion?

We know from computer systems classes as to how function calls are executed on a computer-

- The system maintains a call stack with an `activation record` for each function call. 
- When a function is called, a new activation record is created for the called function that includes the return address (where in the program to return to when the call returns), the values of function call parameters and local variables to the function.
- When a function returns, the control passes back to its caller at the return address stored in the stack.


#### Example- Fibonacci Function

Let us draw the tree for `fibonacci(4)`. 

<img src="http://www.cs.colorado.edu/~srirams/courses/csci3155-fall2018/pictures/fibonacci-tree-4.png" width="75%">
    
## Tail Calls

There is a very special case where the activation records do not have to grow upon successive function calls. These are called *tail calls*.

**Definition (Tail Call)** A function call `f(...)` is said to be a tail call if 
- (a) no further computation is performed when returning from `f` 
- (b) the result is passed back to the caller (without any modifications).

#### Example

Complete below function which calculates sum of first 'N' natural numbers. Natural numbers start from 1,2...N.



In [3]:
import scala.annotation.tailrec

@tailrec
def sumOfNumbersTailRec(n: Int, accum:Int = 0): Int = {
    // Complete the missing portions in this function
    if (n <= 0) { accum }
    else {
        sumOfNumbersTailRec(n-1, accum+n)
    }
}

import scala.annotation.tailrec


defined function sumOfNumbersTailRec

In [4]:
val num1 = 10
val num2 = 5
assert(sumOfNumbersTailRec(num1)==55, "Failed to return 55") // 55 is the correct answer
assert(sumOfNumbersTailRec(num2)==15, "Failed to return 15") // 15 is the correct answer

num1: Int = 10
num2: Int = 5

## Exercise 1

You are given a non recursive function to check if a string is a palindrome. Convert this function to it's tail recursive version.

In [5]:
def checkPalindrome(s: String): Boolean = {
    var start: Int = 0
    var end: Int = s.length-1
    
    while (start < end){
        if (s(start) != s(end)) return false
        start += 1
        end -= 1
    }
    return true
}

defined function checkPalindrome

In [ ]:
// Complete this tail recursive function
import scala.annotation.tailrec

@tailrec
def checkPalindromeTailRec(s:String, start:Int, end:Int): Boolean = {
    
    // Write your code here
    ???
}

In [ ]:
val s1 = "abba"; val s2 = "scala"; val s3 = "malayalam"; val s4 = "aabb"
assert(checkPalindromeTailRec(s1, 0, s1.length - 1)==true, s"Failed. $s1 is a palindrome") // Palindrome
assert(checkPalindromeTailRec(s2, 0, s2.length - 1)==false, s"Failed. $s2 is not a palindrome") // not palindrome
assert(checkPalindromeTailRec(s3, 0, s3.length - 1)==true, s"Failed. $s3 is a palindrome") // palindrome
assert(checkPalindromeTailRec(s4, 0, s4.length - 1)==false, s"Failed. $s4 is not a palindrome") // not palindrome

## Exercise 2
From the scala functions below, find all the tail recursive functions

In [8]:
//option 1-- NOT TAIL

def fun1(x: Int): Int = {
    if (x <= 1)  1
    else fun1(x - fun1(10))
}

// option 2 -- ONLY TAIL 

def fun2(x: Int, y: Int): Int = {
    if (x + y <= 1)  y
    else fun2(x - 2 , y + 1)  
}

// option 3--NOT TAIL

def fun3(x: Int, y: Int): Int = {
    if (x + y <= 1)  y
    else fun3(x - 2 , y + 1) - fun3(x - 3 , y + 2)
}

// option 4--NOT TAIL

def fun4(x: Int, y: Int, z: Int): Int = {
    if (z <= 1)  x - y
    else {
          if (x == y)  fun4( x, y+2, z - 1)
          else fun4( x-2, y-2, fun4(4,4,4) )
    }
}


defined function fun1
defined function fun2
defined function fun3
defined function fun4

# Topic 2- Recurion in Lettuce
### Why do we need special handling for recursion?

If we attempted to define a function such as factorial: 

~~~
let fact = function (x) 
               if (x <= 0) 
               then 1
               else x * fact(x-1)
  in 
    fact(20)
~~~

This will lead to a problem due to the way we have been handling the `let` binding.

Recall that to evaluate a let binding

~~~
let x = <def_expr> in <body_expr>
~~~

under an environment $\sigma$, we proceed in the following steps:
- Evaluate `<def_expr>` under the environment $\sigma$. Let $v$ be the value.
- Evaluate `<body_expr>` under the environment $\sigma[x \mapsto v]$.

The problem is that the identifier $x$ that is being defined is itself not in scope when we are evaluatating `<def_expr>`. The interpreter tries to look up the identifier `fact` and finds that it is not part of the environment.

### Handling Recursion

To handle this, we will need to treat recursive definitions different from regular definitions.

Thus in our new version, we will use a `let rec` construct that tells us that whatever we are going to define is going to be recursive. 

~~~
let rec fact = function (x) 
               if (x <= 0) 
               then 1
               else x * fact(x - 1) 

in 
    fact(10)
~~~

The `let rec` construct is very similar to a let binding but with one big difference: the defining expression for a let rec has to be a function. We will not allow recursive definitions that are not functions. For instance: 

~~~
let rec x = 1 + x
~~~

makes no sense at all, and is disallowed.

Therefore, we will extend Lettuce with the construct

~~~
let rec <function_name_identifier> = function (<param_identifier>)
                                         function_body_expr
                                  in 
                         body_expr
~~~
Let us extend the grammar of a stripped down version:


$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\
 & | & \color{red}{LetRec(\mathbf{Identifier}, \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) } & \text{argument 1 - function name, argument 2 - parameter}\\
 &&& \text{argument 3 - function definition expression, argument 4 - body expr} \\[5pt]
\end{array}$$

For example, we would like to represent the concrete syntax

~~~
let rec f  = function (z) 
                if (0 >= z) then 1 else 1 + f(z - 1)
    in f(10)
~~~

as the following abstract syntax:

~~~
LetRec("f", "z", IfThenElse( 
                         Geq(Const(0), Ident("z")),
                         Const(1),
                         Plus(Const(1), FunCall(Ident("f"), Minus(Ident("z"), Const(1))))
                         ), 
                  FunCall(Ident("f"), Const(10))
         )
~~~

# Handling Recursion using Environments

Another approach to handle recursion is to  directly write an interpreter that handles recursion. However, the problem is that if we are looking at a recursive call:

~~~
let rec f = function (x) <body of function expr> in <def. expr> 
~~~

The main problem to solve is that inside `<body of function expr>` everytime we see a reference to `f`, we should
be able to resolve it to the recursive call itself.  

Let us consider how we handle a normal non-recursive definition

~~~
let f = function (x) <body of function expr> in <def. expr>
~~~

Under enviroment $\sigma$, we evaluate the function body to a closure that stores three things: 
Closure(x, `<body of function expr>`, $\sigma$).

This does not work for a recursive function `f` since when we execute a recursive call to `f` in the
`<body of function expr>`, it is not defined in the environment $\sigma$.

Therefore, the strategy is to extend $\sigma$ into a new environent $\color{red}{\hat{\sigma}}$ as follows:
- $\color{red}{\hat{\sigma}}(x) = \sigma(x)$ for all identifiers $x \not= f$.
- $\color{red}{\hat{\sigma}}(f) = \texttt{Closure}(x, \texttt{<body of function expr>}, \color{red}{\hat{\sigma}})$

I.e, $\color{red}{\hat{\sigma}}$ defines the function $f$ as a closure and notice that 
that the environment of the closure is $\color{red}{\hat{\sigma}}$  again! This means that if we call
$f$ in the body of the function expr, we get the same environment $\hat{\sigma}$ with just the formal parameter
$x$ updated. This gives us what we need to implement recursion: an environment that keeps defining $f$ to the
appropriate function.


Thus, we are going to reconsider environments from scratch?

## Enviroments

What is an environment? We have defined an environment as a function from identifiers to values denoted by them. What sort of environments have we encountered?
- $\texttt{EmptyEnv}$ - The empty environment: implemented as an empty map. Let us call this environment: $\texttt{EmptyEnv}$.
- $\texttt{Extend}(\sigma, x, v)$ - The environment $\sigma[x \mapsto v]$ which denotes a previously existing environment $\sigma$ extended with the mapping $x \mapsto v$ that associates identifier $x$ with value $v$. Let us call this operation 
$\texttt{Extend}(\sigma, x, v)$.

Let us add a third kind of extension to support recursion:
- $\texttt{ExtendRec}(f, x, \texttt{e}, \sigma)$ which creates a new environment $\color{red}{\hat{\sigma}}$ such that $\color{red}{\hat{\sigma}}(x) = \sigma(x)$ for all identifiers $x \not= f$ and $\color{red}{\hat{\sigma}}(f) = \texttt{Closure}(x, \texttt{<body of function expr>}, \color{red}{\hat{\sigma}})$.





## Exercise 3
Consider the recursive function in lettuce

~~~

let rec f = function (x) { 
                   if (x <= 1) 
                   then x
                   else x + f(x - 2)
                    } in 
             f(20)

~~~


Let env denote the environment under which the subexpression f(20) is being evaluated. Select all the facts that are true about this environment.

Let body denote the expression of the body of the function f: "if (x<=1) ... else x + f(x-2)".

Note that we will write circular scopes as `ExtendRec("rec_fun_name", "formal_param_name",  body_expression, old_environment)` where the argument names should be self-explanatory.


Which of the fllowing options are correct?

###### option 1--CORRECT
- env is written `ExtendRec("f", "x", body, EmptyEnvironment)`

###### option 2--INCORRECT x is not a function
- Looking up the identifier `"x"` in env yields `Closure("x", body, env)`

###### option 3--CORRECT
- Looking up the identifier `"f"` in env yields `Closure("x", body, env)`

###### option 4--CORRECT x is not defind 
- The identifier `"x"` is not defined in the environment env.

###### option 5 -- INCORRECT it should be f,x,body,emptyenv
- env is written `ExtendRec("x", "f", body, EmptyEnvironment)`

###### option 6--INCORRECT wrong env, should be updated env
- Looking up the identifier `"f"` in env yields `Closure("x", body, EmptyEnvironment)`


## Exercise 4

Suppose we evaluate the following Lettuce expression starting with the empty environment.
~~~
let y = 20 in  (* let binding for y # 1 *)
  let f = function (x) x + y in 
   let y = 10 in  (* let binding for y # 2 *) 
     let g = function (y) f(y) in 
        g(y)

~~~

Let env be the environment when the function call `g(y)` in the very last line is about to be executed. Match each of the questions below to the correct answer.

This is already defined-
~~~
sealed trait Value
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
case class Closure(x: String, e: Expr, pi: Environment) extends Value
~~~
Comments are written within (* and *) are not part of the expression.
```
What does lookup(env, "y") yield? 

What does lookup(env, "x") yield? 

What does lookup(env, "f") yield? 

The body of the function f refers to identifier "y". What value will this identifier have if function f were called? 

What value does the entire program evaluate to? 

```

- What does lookup(env, "y") yield? 
    - NumValue(10)

- What does lookup(env, "x") yield? 
    - error

- What does lookup(env, "f") yield? 
    - Closure("x", x+y, y->20)

- The body of the function f refers to identifier "y". What value will this identifier have if function f were called? 
    - NumValue(20) because at the time of function defination the value of y is 20 bc static scope

- What value does the entire program evaluate to? 
    - NumValue(30)